In [ ]:
import numpy as np
import random
from torchvision import transforms
import torch

data_path = 'E:\\code_data\\2_Wafer_Map\\Mixed_wafer_failure\\Wafer_Map_Datasets.npz'
data = np.load(data_path)   # npz文件通常是 NumPy 库保存的多个数组的压缩文件，而 pandas 用于处理结构化数据和表格数据
print(data.files)   # 查看npz文件中的数组名字
print(type(data))

In [ ]:
arr0 = data['arr_0']
arr1 = data['arr_1']
# arr0是晶圆图的二维数组矩阵，arr1是缺陷类型的one-hot编码
print(len(arr0),len(arr1))
print(arr0[20],arr1[20])    # 0表示空白点，1表示通过电性测试的正常晶片，2表示电性测试不通过的破碎晶片
print(type(arr0),type(arr1))
print(type(arr0[10]),type(arr1[10]))
Total_Num = len(arr1)     #  下标0到38014

In [ ]:
tensor_arr0 = torch.tensor(arr0, dtype=torch.float32)
tensor_arr1 = torch.tensor(arr1, dtype=torch.float32)

In [ ]:
print(type(tensor_arr0))
print(type(tensor_arr0[0]))
print(tensor_arr0[0])
print(type(tensor_arr1))
print(type(tensor_arr1[0]))
print(tensor_arr1[0])

In [ ]:
# 将晶圆图标签的one-hot编码转换为对应的类别标签
# Center(C),Donut(D),Edge_Loc(EL),Edge_Ring(ER),Loc(L),Near_Full(NF),Scratch(S),Random(R);Normal(N)表示缺陷模式无规律图案
def trans(arr):
    dic = {0:'C',1:'D',2:'EL',3:'ER',4:'L',5:'NF',6:'S',7:'R'}
    dic_complete = {0:'Center',1:'Donut',2:'Edge_loc',3:'Edge_ring',4:'Loc',5:'Nearfull',6:'Scratch',7:'Random'}
    # 单一缺陷我们使用完整的缺陷名，与defect_att中的键名对应一致
    str1 = ''
    if arr.sum() == 0:
         str2 = 'Normal'
    else:
        if arr.sum() == 1:
            for i in range(len(arr)):
                if arr[i] == 1:
                    str2 = dic_complete[i]
                    break
        else:
            for i in range(len(arr)):
                if arr[i] == 1:
                    str1 = str1+dic[i]+'+'
                str2 = str1[:-1]  # 去除最后的加号，这个数组切片的作用是去除最后一个元素
    return str2


In [ ]:
ra = random.randint(0,Total_Num)
print(ra)
print(arr1[ra])
print(trans(arr1[ra]))  # 试验

In [ ]:
label_name = []  #  记录每个晶圆图的标签
for i in range(Total_Num):
    label_name.append(trans(arr1[i]))
for i in range(5):
    r = random.randint(0,Total_Num)
    print(r)
    print(arr1[r])
    print(label_name[r])  # 试验

In [ ]:
print(type(label_name))
label_name = np.array(label_name)
print(type(label_name))

In [ ]:
# 给晶圆图像去噪

# 对于不可变对象，如整数、浮点数、字符串等，函数内部对参数的修改不会影响外部变量的值。
# 这是因为这些对象是通过值传递的，即在函数内部对参数所做的任何修改都仅限于函数内部的副本，而不会改变原始数据‌

# 对于可变对象，如列表、字典等，函数内部对参数的修改会影响到外部变量的值。
# 这是因为这些对象是通过引用传递的，即在函数内部对参数所做的修改直接作用于原始数据，因此在函数外部也可以看到这些修改‌

# 总的来说，Python中的函数参数是否会被修改主要取决于参数的数据类型：
# 不可变对象（如基本数据类型）的参数不会被修改，而可变对象（如列表、字典等）的参数则可能会被修改。
# 这是python语言自己的特性

def denoise(wafermap):
    wm = wafermap.copy()  # wm = wafermap，这样的话，对wm的修改操作也会对wafermap进行修改操作
    # wm2 = wafermap.copy()  # 我们用窗口滑动wm，但修改操作用在wm2上，这样能防止因为消除坏晶粒引起来的连锁反应，会导致晶圆图几乎清空
    # 这就是copy()能够只传值，这样对wm操作就不会影响wafermap。
    row_len = len(wm)  # 获取晶圆图的行数，这里是52
    column_len = len(wm[0])  # 获取晶圆图的列数，这里是52
    threshold = 4/9   # 滤波的阈值
    w_size = 9  # 我们选择3x3的滤波窗口
    for no in range(4):  # 一张晶圆图经过4次滤波效果达到最好，所以这里循环4次
        for i in range(1,row_len-1):  # 我们忽略最边角的晶粒，因为边角的晶粒无法获取完整的滤波窗口，需要进行特殊处理，在此直接忽略不计
            for j in range(1,column_len-1):
                num = 0  # 记录坏晶粒的个数
                if wm[i][j] == 2:  # 只有坏晶粒需要滤波，背景(0)和好晶粒(1)则不需要
                    for row in range(i-1,i+2):
                        for column in range(j-1,j+2):  # 以(i,j)为中心的三行三列
                            if wm[row][column] == 2:
                                num = num + 1
                    result = num/w_size
                    if result < threshold:
                        wm[i][j] = 1
    return wm


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
fig,ax = plt.subplots(nrows=5,ncols=5,figsize=(20,20)) # fig是图框，ax是坐标，三个参数分别是行数，列数，图片大小
ax = ax.ravel(order='C')  # ravel函数的作用是将高维数组转换成一维数组,‘C’ – 按行，‘F’ – 按列，‘A’ – 原顺序，‘k’ – 元素在内存中的出现顺序
# 颜色映射
colors = ['white', 'green', 'red']  # 对应的是0(背景)，1(好晶粒)，2(坏晶粒)
cmap1 = mcolors.ListedColormap(colors)
for i in range(0,21,5):
    j = random.randint(0,Total_Num)
    img = arr0[j]
    img_denoise = denoise(img)
    img_denoise2 = denoise(img_denoise)
    img_denoise3 = denoise(img_denoise2)
    img_denoise4 = denoise(img_denoise3)

    ax[i].imshow(img,cmap=cmap1)
    ax[i].set_title(label_name[j],fontsize=14)   # fontsize是字体大小
    ax[i].set_xlabel(j,fontsize=12)
    ax[i].set_xticks([])  # 设置x轴，此处为空
    ax[i].set_yticks([])  # 设置y轴，此处为空

    ax[i+1].imshow(img_denoise,cmap=cmap1)
    ax[i+1].set_title(f'denoise_{label_name[j]}',fontsize=14)   # fontsize是字体大小
    ax[i+1].set_xlabel(j,fontsize=12)
    ax[i+1].set_xticks([])  # 设置x轴，此处为空
    ax[i+1].set_yticks([])  # 设置y轴，此处为空

    ax[i+2].imshow(img_denoise2,cmap=cmap1)
    ax[i+2].set_title(f'denoise2_{label_name[j]}',fontsize=14)   # fontsize是字体大小
    ax[i+2].set_xlabel(j,fontsize=12)
    ax[i+2].set_xticks([])  # 设置x轴，此处为空
    ax[i+2].set_yticks([])  # 设置y轴，此处为空

    ax[i+3].imshow(img_denoise3,cmap=cmap1)
    ax[i+3].set_title(f'denoise3_{label_name[j]}',fontsize=14)   # fontsize是字体大小
    ax[i+3].set_xlabel(j,fontsize=12)
    ax[i+3].set_xticks([])  # 设置x轴，此处为空
    ax[i+3].set_yticks([])  # 设置y轴，此处为空

    ax[i+4].imshow(img_denoise4,cmap=cmap1)
    ax[i+4].set_title(f'denoise4_{label_name[j]}',fontsize=14)   # fontsize是字体大小
    ax[i+4].set_xlabel(j,fontsize=12)
    ax[i+4].set_xticks([])  # 设置x轴，此处为空
    ax[i+4].set_yticks([])  # 设置y轴，此处为空
plt.tight_layout()  # 紧凑布局，会自动调整子图参数使之填充整个图像区域
plt.show() 

实验发现，一张晶圆图，滤波4次，效果更好

In [ ]:
denoise_wm = []
for i in range(Total_Num):
    denoise_wm.append(denoise(arr0[i]))


In [ ]:
denoise_wm = np.array(denoise_wm)
print(type(denoise_wm))

In [ ]:
np.savez('denoise_WM.npz', original_wm = arr0, label_one_hot = arr1, denoise_wm = denoise_wm, label_name =label_name)

In [ ]:
np.savez('E:\\code_data\\2_Wafer_Map\\Mixed_wafer_failure\\denoise_WM.npz', original_wm = arr0, label_one_hot = arr1, denoise_wm = denoise_wm, label_name =label_name)

In [ ]:
data_path2 = 'E:\\code_data\\2_Wafer_Map\\Mixed_wafer_failure\\denoise_WM.npz'
data2 = np.load(data_path2)   # npz文件通常是 NumPy 库保存的多个数组的压缩文件，而 pandas 用于处理结构化数据和表格数据
print(data2.files)   # 查看npz文件中的数组名字
print(type(data2))

In [ ]:
print(len(data2['label_one_hot']))
print(type(data2['label_one_hot']))